# snowmobile

```{include} /description.md
```

```{eval-rst}

.. toctree::
    :maxdepth: 1
    :hidden:

    ./setup.md

.. toctree::
    :caption: Usage
    :maxdepth: 1
    :hidden:

    ./usage/snowmobile.md
    ./usage/table.md
    ./usage/script.md
    ./usage/sql.md
    ./usage/snowmobile_toml.md

.. toctree::
    :caption: Additional Resources
    :maxdepth: 1
    :hidden:

    ./modindex.md
    ./extensions.md
    ./usage/advanced.md
    ./snippets.md

.. toctree::
    :caption: Meta
    :maxdepth: 1
    :hidden:

    ./acknowledgements.md
    ./changelog.md
    ./authors.md
    ./license.md
```

## Example
- *[Connecting](#connecting)*
- *[Loading Data](#loading-data)*
- *[Executing Queries](#executing-queries)*
- *[Working with SQL Scripts](#working-with-sql-scripts)*
- *[Information API](#information-api)*

<br>

### *Connecting*
<hr class="sn-green-thick sn-indent-h-cell-right-hr">

````{admonition} snowmobile.connect()
:class: toggle, note, is_explanation, sn-dedent-v-container, sn-indent-h-cell-right

```{div} sn-dedent-v-t, sn-dedent-v-b-h
{func}`snowmobile.connect()` is an alias for {class}`snowmobile.Snowmobile()`; its purpose is to:
```
1.  Locate, instantiate, and store [snowmobile.toml](./usage/snowmobile_toml.md#snowmobiletoml)
    as a {class}`~snowmobile.Configuration` object ({class}`sn.cfg`)
1.  Establish a connection to {xref}`snowflake` and store the {xref}`SnowflakeConnection` ({class}`sn.con`)
1.  Serve as the primary entry point to the {xref}`SnowflakeConnection` and {xref}`snowmobile` APIs
+++
```{div} sn-dedent-v-t-h
  The first time it's invoked, [Snowmobile](./usage/snowmobile.md) will find [snowmobile.toml](./usage/snowmobile_toml) and cache its location;
  this step isn't repeated unless the file is moved, the cache is manually cleared, or a new version of {xref}`snowmobile` is installed.
```
+++
**With all arguments omitted, it will authenticate with the default credentials and connection arguments specified in** [**snowmobile.toml**](./usage/snowmobile_toml).

<hr class="sn-green">

{link-badge}`./usage/snowmobile.html,cls=badge-primary badge text-white,Usage: snowmobile.Snowmobile,tooltip=Intro & usage documentation for snowmobile.Snowmobile`
{link-badge}`./autoapi/snowmobile/core/connection/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.connection,tooltip=Documentation parsed from docstrings`
````

```{div} sn-pre-code, sn-dedent-v-t-h
Establishing a connection from configured defaults is done with:
```

In [29]:
import snowmobile

sn = snowmobile.connect()

Locating credentials..
(1 of 2) Finding snowmobile.toml..
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='creds1')


```{div} sn-pre-code, sn-post-code
{meth}`~snowmobile.connect()` returns a [](./usage/snowmobile) with the following attributes:
```

In [6]:
print(sn)            #> snowmobile.Snowmobile(creds='creds1')
print(sn.cfg)        #> snowmobile.Configuration('snowmobile.toml')
print(type(sn.con))  #> <class 'snowflake.connector.connection.SnowflakeConnection'>

snowmobile.Snowmobile(creds='creds1')
snowmobile.Configuration('snowmobile.toml')
<class 'snowflake.connector.connection.SnowflakeConnection'>


```{div} sn-pre-code, sn-post-code
Different sets of connection parameters are accessed by assigned alias:
```

In [36]:
sn2 = snowmobile.connect(creds='sandbox')

print(sn.cfg.connection.current != sn2.cfg.connection.current) #> True

<br>

````{admonition} Fixtures
:class: sn-fixture, toggle, toggle-shown

```{div} hanging
The examples below make use of the `sn` \
created above and this *sample_table*:
```
```{div} sn-dedent-v-b-container
|   COL1 |   COL2 |
|-------:|-------:|
|      1 |      0 |
|      1 |      0 |
|      1 |      0 |
```
````

<hr class="sn-spacer">

<br>

### *Executing Queries*
<hr class="sn-green-thick sn-indent-h-cell-right-hr">

`````{admonition} sn: snowmobile.Snowmobile
:class: toggle, is_explanation, sn-dedent-v-container, sn-indent-h-cell-right

```{div} sn-dedent-t-h, hanging
  {xref}`snowmobile` provides three convenience methods for executing raw SQL directly off `sn`:
```

{meth}`~snowmobile.Snowmobile.query()` implements {meth}`pandas.read_sql()` for querying results into a {class}`~pandas.DataFrame`\
{link-badge}`./autoapi/snowmobile/core/connection/index.html#snowmobile.core.connector.Snowmobile.query,cls=badge-secondary badge-pill text-white,API Docs: Snowmobile.query(),tooltip=Documentation parsed from module docstring`

+++

{meth}`~snowmobile.Snowmobile.ex()` implements {meth}`SnowflakeConnection.cursor().execute()` for executing commands within a {xref}`SnowflakeCursor`\
{link-badge}`./autoapi/snowmobile/core/connection/index.html#snowmobile.core.connector.Snowmobile.ex,cls=badge-secondary badge-pill text-white,API Docs: Snowmobile.ex()   ,tooltip=Documentation parsed from module docstring`

+++

{meth}`~snowmobile.Snowmobile.exd()` implements {meth}`SnowflakeConnection.cursor(DictCursor).execute()` for executing commands within a {xref}`DictCursor`\
{link-badge}`./autoapi/snowmobile/core/connection/index.html#snowmobile.core.connector.Snowmobile.exd,cls=badge-secondary badge-pill text-white,API Docs: Snowmobile.exd(),tooltip=Documentation parsed from module docstring`

+++

<hr class="sn-green">

{link-badge}`./usage/snowmobile.html#executing-raw-sql,cls=badge-primary badge text-white,Usage: Executing Raw SQL,tooltip=Usage documentation for Executing Raw SQL`
{link-badge}`./autoapi/snowmobile/core/connection/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.connection,tooltip=Documentation parsed from docstrings`

`````

`````{admonition} Fixture: **sample_table**
:class: sn-fixture, sn-dedent-v-b-container, toggle, toggle-shown

```{div} hanging
*sample_table* contains:
```
```{div} sn-dedent-v-b-container
|   COL1 |   COL2 |
|-------:|-------:|
|      1 |      0 |
|      1 |      0 |
|      1 |      0 |
```

`````

```{div} sn-pre-code
Into a {class}`~pandas.DataFrame`:
```

In [13]:
sn.query('select * from sample_table')

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


```{div} sn-pre-code
Into a {xref}`SnowflakeCursor`:
```

In [16]:
sn.ex('select * from sample_table').fetchall()

[(1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]

```{div} sn-pre-code
Into a {xref}`DictCursor`:
```

In [20]:
sn.exd('select * from sample_table').fetchall()

[{'COL1': 1.0, 'COL2': 0.0},
 {'COL1': 1.0, 'COL2': 0.0},
 {'COL1': 1.0, 'COL2': 0.0}]

```{div} sn-pre-code
Or into a scalar:
```

In [14]:
sn.query('select count(*) from sample_table', as_scalar=True)

3

<br>

### *Information API*
<hr class="sn-green-thick sn-indent-h-cell-right-hr">

`````{admonition} snowmobile.SQL
:class: toggle, is_explanation, sn-dedent-v-container, sn-indent-h-cell-right

```{div} sn-dedent-v-t
{class}`snowmobile.SQL` generates and executes raw SQL from inputs;
it comes free as the {attr}`~snowmobile.Snowmobile.sql` attribute of
{class}`~snowmobile.Snowmobile`, and its purpose is to provide a simple
Python API to query metadata and execute administrative commands
against {xref}`snowflake`.
```

<hr class="sn-blue">

{link-badge}`./usage/sql.html,cls=badge-primary badge text-white,Usage Docs: snowmobile.SQL,tooltip=Intro & usage documentation for snowmobile.SQL`
{link-badge}`./autoapi/snowmobile/core/sql/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.sql,tooltip=Documentation parsed from docstrings`

`````

<p class="sn-indent-cell"></p>

```{div} sn-pre-code
Check existence of tables/views:
```

In [5]:
sn.sql.exists('sample_table')  #> True

True

```{div} sn-pre-code, sn-post-code
Sample records from a table:
```

In [2]:
sn.sql.table_sample('sample_table', n=-1)

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


```{div} sn-pre-code
Query depth:
```

In [4]:
sn.sql.cnt_records('sample_table')  #> 3

3

```{div} sn-pre-code, sn-post-code
Submit basic administrative commands:
```

In [3]:
sn.sql.clone(nm='sample_table', to='sample_table2')

,status
0,Table SAMPLE_TABLE2 successfully created.


```{div} sn-pre-code, sn-post-code
Fetch columns:
```

In [5]:
sn.sql.columns('sample_table2')  #> ['COL1', 'COL2']

['COL1', 'COL2']

```{div} sn-pre-code, sn-post-code
Query DDL:
```

In [53]:
print(sn.sql.ddl('sample_table'))

create or replace TABLE SAMPLE_TABLE (
	COL1 FLOAT,
	COL2 FLOAT
);


```{div} sn-pre-code
Provide `run=False` to get the raw sql
```

In [7]:
drop_sql = sn.sql.drop('sample_table', run=False)

print(drop_sql)        #> drop table if exists GEM7318.SAMPLE_TABLE

<class 'str'>
drop table if exists GEM7318.SAMPLE_TABLE


```{div} sn-pre-code
Drop objects:
```

In [8]:
for t in ['sample_table', 'sample_table2']:
    sn.sql.drop(t, obj='table')

sn.sql.exists('sample_table')   #> False
sn.sql.exists('sample_table2')  #> False

<br>

### *Loading Data*
<hr class="sn-green-thick sn-indent-h-cell-right-hr">

`````{admonition} snowmobile.Table
:class: toggle, is_explanation, sn-dedent-v-t-container, sn-dedent-v-b-h-container, sn-indent-h-cell-right

````{tabbed} Context
  {class}`~snowmobile.Table` is a loading solution that at minimum accepts a
  [**Snowmobile**](./usage/snowmobile) ({class}`sn`), a {class}`~pandas.DataFrame` ({class}`df`),
  and a table name ({class}`str`).
+++
  In the same way that [**Snowmobile**](./usage/snowmobile) handles its keyword arguments,
  {class}`~snowmobile.Table` will adhere to any arguments explicitly provided and defer
  to the values configured in [snowmobile.toml](./usage/snowmobile_toml) otherwise.
````

````{tabbed} +
  ```{div} sn-dedent-v-b-h, sn-dedent-v-t
  *The behavior outlined below reflects those within the
  [default snowmobile.toml file](./usage/snowmobile_toml.md#file-contents)*, meaning that `t1` will:
  ```

  1. Check if *sample_table* exists in the schema associated with {attr}`sn.con`
  2. If *sample_table* **does** exist, it will validate `df` against *sample_table* and throw an error
     if their dimensions are not identical
  3. If *sample_table* does **not** exist (as is the case here), it will generate DDL from `df` and execute it as part of the loading process
````

<hr class="sn-green">

{link-badge}`./usage/table.html,cls=badge-primary badge text-white,Usage: snowmobile.Table,tooltip=Intro & usage documentation for snowmobile.Table`
{link-badge}`./autoapi/snowmobile/core/table/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.table,tooltip=Documentation parsed from docstrings`
`````

`````{admonition} Setup
:class: toggle, is-setup, sn-clear-title, toggle-shown

````{panels}

The `df` used below is created with:

```python
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {
    'COL1': np.ones(3), 'COL2': np.zeros(3)
    }
)
print(df.shape)  #> (3, 2)
```

---

|   COL1 |   COL2 |
|-------:|-------:|
|      1 |      0 |
|      1 |      0 |
|      1 |      0 |

````

`````

In [16]:
# Verify table does not exist before moving forward with example
if sn.sql.exists('sample_table'):
    sn.sql.drop('sample_table')

In [2]:
"""Generate dummy DataFrame for snowmobile.Table example."""
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {'COL1': np.ones(3), 'COL2': np.zeros(3)}
)
df.head()

,COL1,COL2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


<hr class="sn-spacer">

```{div} sn-pre-code, sn-indent-v-t-container
Given `sn` [Snowmobile](./usage/snowmobile), a `df` ({class}`~pandas.DataFrame`), and a `table` ({class}`str`) to load into, a [**Table**](./usage/table) can be created with:
```

In [17]:
t1 = snowmobile.Table(sn=sn, df=df, table='sample_table')

```{div} sn-pre-code, sn-post-code
With [](./usage/snowmobile_toml) defaults, `t1` has pre-inspected some things like:
```

In [18]:
print(t1.exists)  #> False  -> 'sample_table' doesn't exist in the schema associated with 'sn'

False


```{div} sn-pre-code
We can create *sample_table* and load `df` into with:
```

In [18]:
t1.load()

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds


snowmobile.Table(table='SAMPLE_TABLE')

```{div} sn-pre-code
Here are two ways to verify `t1.load()` did its job:
```

In [19]:
print(t1.loaded)                           #> True -> t1 didn't encounter any errors during .load()
print(sn.sql.cnt_records('sample_table'))  #> 3  ---> 'sample_table' contains three records

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds


snowmobile.Table(table='SAMPLE_TABLE')

```{div} sn-indent-h-cell
<hr class="sn-green-medium" style="margin-top: 1.4rem; margin-bottom: 0.5rem;">
```

```{div} sn-pre-code
In instances where the {class}`~pandas.DataFrame` and table have different dimensions:
```

In [8]:
df2 = pd.concat([df, df], axis=1)
df2.head(1)

,COL1,COL2,COL1,COL2
0,1.0,0.0,1.0,0.0


```{div} sn-pre-code
Here's what `t2` knows about `df` and *sample_table*:
```

In [25]:
t2 = snowmobile.Table(sn=sn, df=df2, table='sample_table')

print(t2.exists)      #> True  --> 'sample_table' exists in the schema associated with `sn`
print(t2.cols_match)  #> False  -> `df2` and 'sample_table' don't share the same dimensions

```{div} sn-pre-code, sn-post-code
`t2` also checked the columns of `df2` and deduped them by appending a suffix to the second set:
```

In [10]:
t3.df.head(1)

,COL1,COL2,COL1_1,COL2_1
0,1.0,0.0,1.0,0.0


```{div} sn-pre-code
Loading in the same way we did `t1` will throw an error with the default [snowmobile.toml](./usage/snowmobile_toml):
```

In [33]:
from snowmobile.core.errors import ColumnMismatchError

try:
    t3.load()
except ExistingTableError as e:
    print(e)
"""
ColumnMismatchError: `SAMPLE_TABLE` columns do not equal those in the local DataFrame 
and if_exists='append' was specified.
Either provide if_exists='replace' to overwrite the existing table or see `table.col_diff`
to inspect the mismatched columns.
"""

ColumnMismatchError: `SAMPLE_TABLE` columns do not equal those in the local DataFrame and if_exists='append' was specified.
Either provide if_exists='replace' to overwrite the existing table or see `table.col_diff` to inspect the mismatched columns.

```{div} sn-pre-code, sn-post-code
Explicit arguments take precedent over configurations in [snowmobile.toml](./usage/snowmobile_toml), so `df2` *could* be loaded with:
```

In [15]:
# t3.load(if_exists='replace')

<br>

```{admonition} Fixture: **sample_table**
:class: toggle, toggle-shown, sn-fixture
The examples below make use of *sample_table* created here.
```

<hr class="sn-spacer">

<br>

### *Working with SQL Scripts*
<hr class="sn-green-thick sn-indent-h-cell-right-hr">

`````{admonition} snowmobile.Script
:class: toggle, is_explanation, sn-dedent-v-t-container, sn-indent-h-cell-right

```{div} sn-dedent-v-t-h, sn-dedent-v-b-h
[**snowmobile.Script**](./usage/script.ipynb) imports a sql file and parses its contents according to its structure.
```

At a minimum, the file is split into individual statements, each of which is checked for tags;
if none are found, [Script](./usage/script.ipynb) will generate a generic name for the statement based on
the literal first SQL keyword it contains and its index position (e.g. `select data~statement #3` below).

+++

```{div} sn-dedent-v-b-h
By providing {class}`script` (below) the same instance of {class}`sn` with which {class}`t1` (above)
was instantiated, **the {xref}`SnowflakeConnection` and [Configuration](./usage/snowmobile_toml) is
shared amongst:**
```

- {class}`sn:` {class}`~snowmobile.Snowmobile`
- {class}`t1:` {class}`~snowmobile.Table`
- {class}`script:` {class}`~snowmobile.Script`


<hr class="sn-green">

{link-badge}`./usage/script.html,cls=badge-primary badge text-white,Usage: snowmobile.Script,tooltip=Intro & usage documentation for snowmobile.Script`
{link-badge}`./autoapi/snowmobile/core/script/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.script,tooltip=Documentation parsed from docstrings`

`````

````{admonition} Setup
:class: toggle, is-setup, sn-clear-title, toggle-shown

```{div} sn-dedent-v-t-h, hanging
The `path` used below is a full path ({class}`str` or {class}`pathlib.Path`) \
to **sample_table.sql**:
```

```{literalinclude}  ./snippets/getting_started/sample_table.sql
:language: sql
:lines: 1-17
```

<hr class="sn-spacer-thin">

````

In [21]:
# Setup
from pathlib import Path

path = Path.cwd() / 'snippets' / 'getting_started' / 'sample_table.sql'

```{div} sn-pre-code
Given a `path` to *sample_table.sql*, a [Script](./usage/script) can be created with:
```

In [31]:
script = snowmobile.Script(sn=sn, path=path)

print(script)  #> snowmobile.Script('sample_table.sql')

snowmobile.Script('sample_table.sql')


```{div} sn-pre-code
`.dtl()` prints a high level view of contents to stdout:
```

In [13]:
script.dtl()

sample_table.sql
1: Statement('create transient table~any_random_table1')
2: Statement('create table~sample_table')
3: Statement('select data~statement #3')
4: Statement('select data~sample_table')


```{div} sn-pre-code
Individual statements can be accessed by index position:
```

In [14]:
script(4)   #> Statement('select data~sample_table')
script(-1)  #> Statement('select data~sample_table')

Statement('select data~sample_table')

```{div} sn-pre-code
Or by their tag as a string:
```

In [16]:
script('select data~sample_table') #> Statement('select data~sample_table')

Statement('select data~sample_table')

```{div} sn-pre-code
They can be run directly off the Script:
```

In [33]:
script.run(4)
script(4).results.head(1)

,col1,col2
0,1.0,0.0


```{div} sn-pre-code
Or stored and worked with independently:
```

In [32]:
s4 = script(4).run()
s4.results.head(1)

,col1,col2
0,1.0,0.0


```{div} sn-pre-code
See [Advanced Examples](./usage/advanced.md) for more in-depth applications of {class}`~snowmobile.Script`.
```

# <u>Meta / Non-Output</u>
---

All cells below this are either excluded from output via the `remove-cell` cell-tag
or contain contents that will not visibly render in the output.

# Style


```css
<style>
.md-typeset h1, .md-typeset h2 {
    margin-top: -0.5rem;
}
    
.md-typeset h3 {
    margin-top: -0.5rem;
    margin-bottom: 0.2rem;
    font-size: 140%;
}
    
.md-typeset .admonition.is-setup {
    border-left-color: #11838e;
    margin: -0.15rem 1.5rem .7rem 0.85rem;
<!--     display: block; -->
}
</style>
```

<style>
.md-typeset h1, .md-typeset h2 {
    margin-top: -0.5rem;
}
    
.md-typeset h3 {
    margin-top: -0.5rem;
    margin-bottom: 0.2rem;
    font-size: 140%;
}
    
.md-typeset .admonition.is-setup {
    border-left-color: #11838e;
    margin: -0.15rem 1.5rem .7rem 0.85rem;
<!--     display: block; -->
}
</style>